In [ ]:
import requests
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup as BS
import re
import pandas as pd
import time
from dateutil.parser import parse

In [ ]:
madison_garden = ('https://www.concertarchives.org/venues/madison-square-garden--4?page=REPLACE#concert-table')
united_center = ('https://www.concertarchives.org/venues/united-center--2?page=REPLACE#concert-table')
the_fillmore = ('https://www.concertarchives.org/venues/the-fillmore?page=REPLACE#concert-table')
club_quattro = ('https://www.concertarchives.org/venues/club-quattro-074f58c1-a434-4e63-9efa-80dae7c28dda?page=REPLACE#concert-table')
columbiahalle = ('https://www.concertarchives.org/venues/columbiahalle?page=REPLACE#concert-table')
opera_house = ('https://www.concertarchives.org/venues/concert-hall-sydney-opera-house--4?page=REPLACE#concert-table')
rock_city = ('https://www.concertarchives.org/venues/rock-city?page=REPLACE#concert-table')

In [ ]:
name = ['Madison Square Garden', 'United Center', 'The Fillmore', 'Club Quattro', 'Columbiahalle', 'Concert Hall, Sydney Opera House', 'Rock City']
location = ['New York, New York, United States', 'Chicago, Illinois, United States', 'Detroit, Michigan, United States', 'Osaka, Ōsaka, Japan', 'Berlin, Berlin, Germany', 'Sydney, New South Wales, Australia', 'Nottingham, England, United Kingdom']
url = [madison_garden, united_center, the_fillmore, club_quattro, columbiahalle, opera_house, rock_city]
country = ['America', 'America', 'America', 'Japan', 'Germany', 'Australia', 'United Kingdom']
pages = [20, 11, 9, 1, 11, 4, 15]

halls = {'name': name, 'location':location, 'url':url, 'country':country, 'pages':pages}
halls_df = pd.DataFrame(halls)

In [ ]:
halls_df

In [ ]:
def GetRequest (url):
    req = Request(url , headers={'User-Agent': 'Mozilla/5.0'})
    webpage = urlopen(req).read()
    page_soup = BS(webpage, "html.parser")
    str_page = page_soup.findAll("span", class_="")
    inner_soup = BS(str(str_page), "html.parser")
    spans = inner_soup.findAll("span", class_="")
    
    return spans

In [ ]:
def is_date(string, fuzzy=False):
    try:
        parse(string, fuzzy=fuzzy)
        return True
    except ValueError:
        return False

def CleanHTML(spans):
    res = str(spans)
    res = re.sub('<span>', '', res)
    res = re.sub('</span>', '', res)
    res = re.sub('<strong>', '', res)
    res = re.sub('</strong>', '', res)
    res = re.sub('<span class="">', '', res)
    res = re.sub('\n', '', res)
    res = re.sub('</a>', '', res)
    res = re.split(">", res)[-1]
    
    return res

In [ ]:
year = []
artist = []
venue = []
location = []
country = []

main_index = 0
#new_index = 0
for index, row in halls_df.iterrows():
    for x in range(row['pages']):
        main_index += 1
        time.sleep(1)
        spans = GetRequest(row['url'].replace('REPLACE', str(main_index)))
        #print(spans)
        new_index = 0

        for span in spans:
            #print(span)
            new_index += 1
            if new_index == 1:
                year += [(CleanHTML(span))] 
                #print(CleanHTML(span))
            elif new_index == 2:
                artist += [(CleanHTML(span))]
                #print(CleanHTML(span))
            elif new_index == 3:
                #print(CleanHTML(span))
                venue += [(CleanHTML(span)).replace('…', row['name'])]
                #print(CleanHTML(span))
            elif new_index == 4: 
                if is_date(CleanHTML(span)) == True:
                    location.append(row['location'])
                    year += [(CleanHTML(span))]
                    new_index = 1
                    country += [row['country']]
                else:
                    location += [(CleanHTML(span))]
                    country += [row['country']]
               # print(CleanHTML(span))
                    new_index = 0
            
            
    venues = pd.DataFrame(list(zip(year, artist, venue, location, country)), 
                               columns =['year', 'artist', 'venue', 'location', 'country'])

In [ ]:
pd.set_option('display.max_rows', None)

venues = venues.drop_duplicates(keep='last')

venues

#pd.reset_option('all')

In [ ]:
#venues.to_csv('venues.csv')
#venues.to_csv('C:/Users/natha/Documents/NSS/Projects/capstone_3/data/venues.csv')